In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

#other computational libraries
import numpy as np
import pandas as pd
import scipy
import scipy.constants as co
import scipy.stats as ss
import itertools
import pickle

In [3]:
import flux_recoil_spec as frs
import importlib

In [13]:
importlib.reload(frs)

frs.SNOLAB_flux()

fast flux: 4000.0000000000005 n/m^2/day
